In [2]:
#Import Libraries
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [74]:
#Extract Job Title Function
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [75]:
#Extract Job Description
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll("span", attrs={"class": "summary"})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)

In [73]:
#Import the Job Title Dataset
job_titles=pd.read_csv(r"C:\Users\georg\Dropbox\Business Analytics\Spring-Quarter\Big_Data_Content\Project\Git_code\Datasets\Job_Titles.csv",sep=",")

#Create job_titles list
job_list=list(job_titles.Job_titles)

#Replace spaces with '+'
job_list2=[]
for item in job_list:
    job_list2.append(item.replace(' ', '+'))
    
job_list=job_list2

In [76]:
#Testing the above code
URL ="https://www.indeed.com/jobs?q=data%2420%2C000&l=+New+York&start=10"
    
#conducting a request of the stated URL above:
page = requests.get(URL)

#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")

extract_job_title_from_result(soup)
extract_summary_from_result(soup)

['Data management and analysis, 65%. Under direct supervision of the Principal Investigator and senior research team, the incumbent will be responsible for data...',
 'They ensure that product data becomes and remains clean and accurate. The Data Quality Analyst is responsible for structuring and management of product...',
 'Reconciling data discrepancies and inconsistencies. Validating information by cross-referencing data on multiple sources....',
 'Familiarity with various social media platforms and an interest in analyzing/organizing/interpreting data. The Communications Assistant reports to the Media...',
 'We have an opening for a full-time CLERK, DATA ENTRY position. Fills in for other Data Entry Clerks as needed and works cooperatively with others....',
 'Provide technical expertise to the Manager, SRC Data Analysis regarding data models, database design development, and data mining....',
 'Coordinate and collect input data from different departments, check data integrity, summ

In [ ]:
#Main scraper code